# IMF Test

Test the IMF scraper.

In [1]:
%cd ..

C:\Users\ROSA_L\PycharmProjects\scraper


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [4]:
from scraper.core import factory
job = factory.get_scraper_job('org_imf', 'weo')
job.run(download=False)

2021-10-14 21:42:26,111 - scraper.jobs.org_imf.weo - INFO - Getting the links to the files to download...
2021-10-14 21:42:59,646 - scraper.core.utils - INFO - download_and_get_checksum: 18.04327964782715 ms
2021-10-14 21:42:59,686 - scraper.core.utils - INFO - rm_sources_up_to_date: 39.63470458984375 ms
2021-10-14 21:43:00,446 - scraper.jobs.org_imf.weo - INFO - Starting transform()
2021-10-14 21:43:00,447 - scraper.jobs.org_imf.weo - INFO - Starting update_details()
2021-10-14 21:43:00,993 - scraper.jobs.org_imf.weo - INFO - 1190 rows read from org_imf_WEO_WEOOct2021-SDMX-DSD.xlsx
2021-10-14 21:43:01,001 - scraper.jobs.org_imf.weo - INFO - 148 rows after removing NAs.
2021-10-14 21:43:25,689 - root - INFO - 0 items sent to IEA External DB API instance at: http://vipenta.iea.org:8000/dimension/detail
2021-10-14 21:43:25,689 - scraper.jobs.org_imf.weo - INFO - Starting _get_data_df()
2021-10-14 21:43:25,904 - scraper.jobs.org_imf.weo - INFO - 8823 rows read from C:\Users\ROSA_L\Pycharm

In [9]:
[vars(s) for s in job.sources]

[]

In [5]:
vars(imf)

{'full_load': None,
 'dynamic_dim': defaultdict(list, {}),
 'data': None,
 'sources': ['BaseSource(c...r2021all.txt)'],
 'source_complements': ['BaseSource(c...dmxdata.xlsx)']}

In [ ]:
import pandas as pd
from pathlib import Path
path = Path(r"C:\Users\LYNGOPOULOU_D\Pycharm Projects\scraper\filestore\org_imf_WEO_WEOApr2021all.txt")
df = pd.read_csv(path, sep='\t', encoding='ISO-8859-1')
if df.isna().values.all():
    print("OK")
df = df = pd.read_csv(path, sep='\t', encoding='utf-16-le')
df.head()

In [9]:
vars(imf)

{'dynamic_dim': defaultdict(list,
             {'detail': [{'code': 'IMF_SDMX_BCA',
                'category': 'IMF_WEO',
                'description': 'Current account balance',
                'json': {'mapping': defaultdict(list, {})}},
               {'code': 'IMF_SDMX_BCA_NGDPD',
                'category': 'IMF_WEO',
                'description': 'Current account balance',
                'json': {'mapping': defaultdict(list, {})}},
               {'code': 'IMF_SDMX_BF',
                'category': 'IMF_WEO',
                'description': 'Financial account balance',
                'json': {'mapping': defaultdict(list, {})}},
               {'code': 'IMF_SDMX_BFD',
                'category': 'IMF_WEO',
                'description': 'Direct investment, net',
                'json': {'mapping': defaultdict(list, {})}},
               {'code': 'IMF_SDMX_BFF',
                'category': 'IMF_WEO',
                'description': 'Financial derivatives, net',
                'j

## Attempt to stop inferring file links

Attempt to get file URLs directly from the website with BeautifulSoup.

In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = r'https://www.imf.org/en/Publications/SPROLLs/world-economic-outlook-databases#sort=%40imfdate%20descending'

response = requests.get(url)

response.raise_for_status()

soup = BeautifulSoup(response.content, 'lxml')

In [10]:
soup.find('a')

<a name="top"></a>

It did not work with beautiful soup because of the JavaScripts in the page.
Time to try with selenium.

In [1]:
cd ..

C:\Users\ROSA_L\PycharmProjects\scraper


In [15]:
from scraper.jobs.utils import get_driver

url = r'https://www.imf.org/en/Publications/SPROLLs/world-economic-outlook-databases#sort=%40imfdate%20descending'

driver = get_driver()
driver.get(url)

In [16]:
# Get the latest publication link
try:
    publication_container = driver.find_element_by_class_name('coveo-result-list-container')
    year = publication_container.find_element_by_tag_name('h3').text
    print(f'year: {year}')
    latest_publication = driver.find_element_by_class_name('CoveoResult')
    a = latest_publication.find_element_by_tag_name('a')
    print(f'link text: {a.text.split(",")[1]}')
    pub_url = a.get_attribute('href')
    print(pub_url)
    # got to the latest publication page
    new_page = a.click()
except Exception:
    print('Exception!')
finally:
    driver.close()

year: 2021
link text:  April 2021
https://www.imf.org/en/Publications/WEO/weo-database/2021/April


In [4]:
# Find the link to the 'Entire Dataset' page and click on it
entire_dataset = driver.find_element_by_link_text('Entire Dataset')
entire_dataset.click()

In [5]:
print(driver.current_url)

https://www.imf.org/en/Publications/WEO/weo-database/2021/April/download-entire-database


In [6]:
# find the following links
# - By Countries
# - SDMX Data

link_labels = ['By Countries', 'SDMX Data Structure Definition']

# for label in link_labels.keys():
#    link = driver.find_element_by_link_text(label).get_attribute('href')
#    print(f'{label}: {link}')
#    link_labels[label] = link

link_labels2 = {label: driver.find_element_by_link_text(label).get_attribute('href') for label in link_labels}
    
print(link_labels2)



{'By Countries': 'https://www.imf.org/-/media/Files/Publications/WEO/WEO-Database/2021/WEOApr2021all.ashx', 'SDMX Data Structure Definition': 'https://www.imf.org/-/media/Files/Publications/WEO/WEO-Database/2021/weoapr2021-sdmx-dsd.ashx'}


In [12]:
link_labels = {'By Countries': None, 'SDMX Data': None}


['By Countries', 'SDMX Data']

In [21]:
from datetime import datetime

date = datetime.strptime('April 2021', '%B %Y')
date

datetime.datetime(2021, 4, 1, 0, 0)

In [22]:
date.strftime('%b%Y')

'Apr2021'

In [28]:
import pandas as pd
df = pd.read_excel(r"C:\Users\ROSA_L\Downloads\weoapr2021-sdmx-dsd.xlsx", sheet_name='CONCEPT', usecols=[0, 1], skiprows=7)

In [29]:
df.tail()

,Code,Description
1185,Z44C,MA: Derivatives (residual maturity greater tha...
1186,Z44D,MA: Derivatives (residual maturity greater tha...
1187,Z46,MA: Currency composition of reserves (by group...
1188,Z46A,MA: Currency composition of reserves (by group...
1189,Z46B,MA: Currency composition of reserves (by group...
